In [17]:
import pandas as pd

In [18]:
matches = pd.read_csv('nba_matches_indexed.csv', index_col=0)

In [19]:
matches["date"] = pd.to_datetime(matches["Date"])

In [20]:
matches["venue_code"] = matches["Venue"].astype("category").cat.codes

In [21]:
matches["opp_code"] = matches["Opponent"].astype("category").cat.codes

In [22]:
matches["hour"] = matches["StartET"].str.replace(":.+", "", regex=True).astype("int")

In [23]:
matches["day_code"] = matches["date"].dt.dayofweek

In [24]:
matches["target"] = (matches["Result"] == "W").astype("int")

In [25]:
from sklearn.ensemble import RandomForestClassifier

In [26]:
rf = RandomForestClassifier(n_estimators=50, min_samples_split=10, random_state=1)

In [27]:
training_set = matches[matches["date"] < '2024-04-01']

In [28]:
test_set = matches[matches["date"] >= '2024-04-01']

In [29]:
predictors = ["venue_code", "opp_code", "hour", "day_code"]

In [30]:
rf.fit(training_set[predictors], training_set["target"])

ValueError: Found array with 0 sample(s) (shape=(0, 4)) while a minimum of 1 is required by RandomForestClassifier.